In [1]:
"""
Purpose:
Now that have fully optimmized classifier, want to run it on test neurons to make sure runs all the way through
on a couple before I deploy it to Kubernetes
"""

'\nPurpose:\nNow that have fully optimmized classifier, want to run it on test neurons to make sure runs all the way through\non a couple before I deploy it to Kubernetes\n'

# Generating the Labels for each of the test neurons

In [2]:
from OPTIMIZED_whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron as op_extract
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron as unop_extract
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time

dj.config["database.host"] = "10.28.0.34"
dj.config["database.user"] = "celiib"
dj.config["database.password"] = "newceliipass"
dj.config["safemode"] = True
dj.config["display.limit"] = 20

In [3]:
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")

Connecting celiib@10.28.0.34:3306


In [4]:
key = dict(segmentation=3,segment_id=648518346349471910)
#key = dict(segmentation=3,segment_id=648518346349475536)


missing_apical_ids = [648518346349478700,648518346349485870]
not_missing_apicals = [648518346349473813,648518346349475510,648518346349473597] #but both have apicals
total_ids_to_test = missing_apical_ids + not_missing_apicals

In [5]:
#make the current key to use
index = 4
key = dict(segmentation=3,segment_id=total_ids_to_test[index])

In [ ]:
""" Do the labels from theh older version """
start_time = time.time()

print("\n\n*****Starting Auto Labeling for " + str(key["segment_id"]) + "******")

segment_id = key["segment_id"]

#get the vertices and faces from datajoint
# get the newly stitched mesh
# get the original mesh


verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & key).fetch1("vertices","triangles")

cgal_location = "/notebooks/Pass_2_Excitatory_4_Auto_Classifier_Whole_Neuron_Run_2/automatic_classifier_revised_efficiency/temp/"
cgal_file_sdf = str(key["segment_id"]) + "_fixed-cgal_3_0.20_sdf.csv"
cgal_file_seg = str(key["segment_id"]) + "_fixed-cgal_3_0.20.csv"

#run the whole algorithm on the neuron to test
verts_labels, faces_labels = unop_extract(import_Off_Flag=False,segment_id=segment_id,vertices=verts,
                     triangles=faces,pymeshfix_Flag=False,
                     import_CGAL_Flag=True,
                     import_CGAL_paths = [cgal_location + cgal_file_seg,
                                         cgal_location + cgal_file_sdf],
                     return_Only_Labels=True,
                     clusters=3,
                     smoothness=0.2)

print(f"Finished Auto Labeling: {time.time() - start_time}")


In [ ]:
# import importlib
# from OPTIMIZED_whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron as op_extract
# importlib.reload(op_extract)


In [6]:
""" Do the labels from theh older version """
start_time = time.time()

print("\n\n*****Starting Auto Labeling for " + str(key["segment_id"]) + "******")

segment_id = key["segment_id"]

#get the vertices and faces from datajoint
# get the newly stitched mesh
# get the original mesh


verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & key).fetch1("vertices","triangles")

cgal_location = "/notebooks/Pass_2_Excitatory_4_Auto_Classifier_Whole_Neuron_Run_2/automatic_classifier_revised_efficiency/temp/"
cgal_file_sdf = str(key["segment_id"]) + "_fixed-cgal_3_0.20_sdf.csv"
cgal_file_seg = str(key["segment_id"]) + "_fixed-cgal_3_0.20.csv"

#run the whole algorithm on the neuron to test
verts_labels_opt, faces_labels_opt = op_extract(import_Off_Flag=False,segment_id=segment_id,vertices=verts,
                     triangles=faces,pymeshfix_Flag=False,
                     import_CGAL_Flag=True,
                     import_CGAL_paths = [cgal_location + cgal_file_seg,
                                         cgal_location + cgal_file_sdf],
                     return_Only_Labels=True,
                     clusters=3,
                     smoothness=0.2,
                     classifier_stub_threshold_apical=1500)

print(f"Finished Auto Labeling: {time.time() - start_time}")




*****Starting Auto Labeling for 648518346349473597******
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.18613028526306152
2) Staring: Generating CGAL segmentation for neuron
2) Finished: Generating CGAL segmentation for neuron: 38.78980588912964
3) Staring: Generating Graph Structure and Identifying Soma
soma_index = 3
3) Finished: Generating Graph Structure and Identifying Soma: 0.8858537673950195
5) Staring: Finding Apical Index
Soma Index = 3
Soma Connections = [8, 10, 11, 17, 15, 27, 85, 69, 44, 49]
soma_80_percent = 171898.575
possible_Axons_filter_1 = [8, 85]
size_threshold_filter = [8]
height_threshold_filter = [8]
sdf_threshold_filter = [8, 85]
possible_Axons_filter_2 = [8]
possible_Apical = 8
5) Finished: Finding Apical Index: 1.7327632904052734
6) Staring: Classifying Entire Neuron
Neighbor = 17
mean_convex == 25.338650838745842
std_dev_conves == 72.30447029482141
Neighbor = 15
mean_con

# COMPUTING THE CONFUSION MATRIX FOR BOTH OF THE OPTIONS

In [ ]:
actual_labels = (pinky.ProofreadLabel & key).fetch1("triangles")

In [ ]:
#make sure they are all the same size
print(len(faces_labels),len(faces_labels_opt),len(actual_labels))

In [ ]:
import celii_confusion_plotting as cp

In [ ]:
Label_key_data = pinky.LabelKey.fetch(as_dict=True)
kept_data = Label_key_data[2:9] + Label_key_data[10:13]
kept_data

classes = [k["description"] for k in kept_data]
cm_labels = [k["numeric"] for k in kept_data]
classes.append("unknown")
cm_labels.append(13)

In [ ]:
cm_labels

In [ ]:
cp.plot_confusion_matrix_from_raw_data(y_true=actual_labels,
                                       y_pred=faces_labels_opt,
                                       classes=classes,
                                       cm_labels=cm_labels)

In [ ]:
#how to print out the actual labels
#export the labels
labels_opt = faces_labels_opt.astype("int")
np.savetxt("temp/" + str(key["segment_id"]) + '_optimized_output_labels.csv', labels_opt, fmt='%d', delimiter='\n')

In [ ]:
#change all of the labels that are above allowed value to unknown
faces_labels[faces_labels>=13] = 13

from collections import Counter
Counter(faces_labels)

In [ ]:
#change all of the labels that are above allowed value to unknown
faces_labels[faces_labels>=13] = 13

cp.plot_confusion_matrix_from_raw_data(y_true=actual_labels,
                                       y_pred=faces_labels,
                                       classes=classes,
                                       cm_labels=cm_labels)

In [ ]:
#how to print out the actual labels
#export the labels
labels = faces_labels.astype("int")
np.savetxt("temp/" + str(key["segment_id"]) + 'NOT_optimized_output_labels.csv', labels, fmt='%d', delimiter='\n')